# Project 4 Intro to Machine Learning - Identify Fraud from Enron Email - Nitin Ramchand Lalwani

## Introduction

Explain what POI is and the enron scandal overview


## Dataset and Question

### Data Exploration

In [3]:
import pickle
def open_dataset():
    with open("C:\Users\RAMCHAND_N\Data Analytics Nanodegree\CORE_Module4_IntrotoMachineLearning\CORE_Module4_Intro_to_Machine_Learning_NitinRamchandLalwani\Lesson 2- Naive Bayes\\ud120-projects-master\\ud120-projects-master\\final_project\\final_project_dataset.pkl", "r") as data_file:
        data_dict = pickle.load(data_file)
    return data_dict

### Task 2: Remove outliers
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.


my_dataset = open_dataset()

The dataset has the following structure when loaded. It is a python dictionary with people's names as keys and another dictionary as values. In this dictionary, there are all the following features which will be used for the analysis. 

In [23]:
from pprint import pprint
pprint(my_dataset['METTS MARK'])

{'bonus': 600000,
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'mark.metts@enron.com',
 'exercised_stock_options': 'NaN',
 'expenses': 94299,
 'from_messages': 29,
 'from_poi_to_this_person': 38,
 'from_this_person_to_poi': 1,
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 1740,
 'poi': False,
 'restricted_stock': 585062,
 'restricted_stock_deferred': 'NaN',
 'salary': 365788,
 'shared_receipt_with_poi': 702,
 'to_messages': 807,
 'total_payments': 1061827,
 'total_stock_value': 585062}


The total number of people we have in the dataset are:

In [19]:
len(my_dataset)

146

As we can see below there are in total 18 names that have a Person of Interest (POI) tag which comes are the ones containing a True value in for the "POI" feature. This are the number of POIs we have in the dataset, and in total we have 146 people so it means that defining a classification algorithm, we must remember that it is not balanced.

In [9]:
sum(poi)

18

In the POI_names.txt file however we have a total of 35 POIs. 

In [27]:
with open("C:\Users\RAMCHAND_N\Data Analytics Nanodegree\CORE_Module4_IntrotoMachineLearning\CORE_Module4_Intro_to_Machine_Learning_NitinRamchandLalwani\Lesson 2- Naive Bayes\\ud120-projects-master\\ud120-projects-master\\final_project\\poi_names.txt") as f:
    print len(f.readlines()[2:])

35


Now we would like to understand which how much information we have for each of the features for each of the 146 people in our dataset. So some values inside the dataset are shown as "NaN" and we are looking for the percetages of information in the dataset that are not "NaN"

In [60]:
import collections
from pprint import pprint
from __future__ import division
populated_data_count = collections.defaultdict(int)
populated_data_percentage = {}
features = next(my_dataset.itervalues()).keys()
for person_features in my_dataset.itervalues():
    for feature in features:
        if person_features[feature] != "NaN":
            populated_data_count[feature] += 1
for k,v in populated_data_count.iteritems():
    populated_data_percentage[k] = round((v/len(my_dataset)),4)
#pprint(populated_data_count)
pprint(populated_data_percentage)

{'bonus': 0.5616,
 'deferral_payments': 0.2671,
 'deferred_income': 0.3356,
 'director_fees': 0.1164,
 'email_address': 0.7603,
 'exercised_stock_options': 0.6986,
 'expenses': 0.6507,
 'from_messages': 0.589,
 'from_poi_to_this_person': 0.589,
 'from_this_person_to_poi': 0.589,
 'loan_advances': 0.0274,
 'long_term_incentive': 0.4521,
 'other': 0.637,
 'poi': 1.0,
 'restricted_stock': 0.7534,
 'restricted_stock_deferred': 0.1233,
 'salary': 0.6507,
 'shared_receipt_with_poi': 0.589,
 'to_messages': 0.589,
 'total_payments': 0.8562,
 'total_stock_value': 0.863}


From this first exploration, we see that there are some features that don't have much data so lets hihglight the ones that have less than 50% of the people filled which are the following:

In [64]:
for k, v in populated_data_percentage.iteritems():
    if v < 0.5:
        pprint(k)
        

'deferral_payments'
'restricted_stock_deferred'
'deferred_income'
'loan_advances'
'long_term_incentive'
'director_fees'


As a further step, we look at whether from these 6 features without a lot of infomration, whether any of them have information for a lot of POIs at the same time since this may be auseful hint to investigate. 

In [65]:
poi= []
deferral_payments_and_poi = []
restricted_stock_deferred_and_poi = []
deferral_income_and_poi = []
loan_advances_and_poi = []
long_term_incentive_and_poi = []
director_fees_and_poi = []
for person in my_dataset:
    restricted_stock_deferred_and_poi.append((my_dataset[person]['restricted_stock_deferred'] != "NaN") & (my_dataset[person]['poi'] == True))
    poi.append(my_dataset[person]['poi'] == True)
    deferral_payments_and_poi.append((my_dataset[person]['deferral_payments'] != "NaN") & (my_dataset[person]['poi'] == True) )
    deferral_income_and_poi.append((my_dataset[person]['deferred_income'] != "NaN") & (my_dataset[person]['poi'] == True))
    loan_advances_and_poi.append((my_dataset[person]['loan_advances'] != "NaN") & (my_dataset[person]['poi'] == True))
    long_term_incentive_and_poi.append((my_dataset[person]['long_term_incentive'] != "NaN") & (my_dataset[person]['poi'] == True))
    director_fees_and_poi.append((my_dataset[person]['director_fees'] != "NaN") & (my_dataset[person]['poi'] == True))
print "deferral_income_and_poi" , sum(deferral_income_and_poi)
print "loan_advances_and_poi" ,sum(loan_advances_and_poi)
print "restricted_stock_deferred_and_poi" ,sum(restricted_stock_deferred_and_poi)
print "deferral_payments_and_poi" , sum(deferral_payments_and_poi)
print "long_term_incentive_and_poi" ,sum(long_term_incentive_and_poi)
print "director_fees_and_poi" ,sum(director_fees_and_poi)





deferral_income_and_poi 11
loan_advances_and_poi 1
restricted_stock_deferred_and_poi 0
deferral_payments_and_poi 5
long_term_incentive_and_poi 12
director_fees_and_poi 0


We see that for the deferral_income and long_term_incentive features there is not much information overall (for less than 50% of people in the dataset) but there is significantly high percentage of data for those people that have a __True__ tag in POI, so we can discard the other four features but we can keep these two just incase they provide us good information for the classification algorithms.

### Outlier Removal

## Feature Selection

Code just in case for future but not part of the report

In [18]:

email_address_and_poi = []
exercised_stock_options_and_poi = []
expenses_and_poi = []
from_messages_and_poi = []
from_poi_to_this_person_and_poi = []
from_this_person_to_poi_and_poi = []
other_and_poi = []
salary_and_poi = []
shared_receipt_with_poi_and_poi = []
to_messages_and_poi = []
total_payments_and_poi = []
total_stock_value_and_poi = []
bonus_and_poi = []
for person in my_dataset:
    email_address_and_poi.append((my_dataset[person]['email_address'] != "NaN") & (my_dataset[person]['poi'] == True))
    exercised_stock_options_and_poi.append((my_dataset[person]['exercised_stock_options'] != "NaN") & (my_dataset[person]['poi'] == True))
    expenses_and_poi.append((my_dataset[person]['expenses'] != "NaN") & (my_dataset[person]['poi'] == True))
    from_messages_and_poi.append((my_dataset[person]['from_messages'] != "NaN") & (my_dataset[person]['poi'] == True))
    from_poi_to_this_person_and_poi.append((my_dataset[person]['from_poi_to_this_person'] != "NaN") & (my_dataset[person]['poi'] == True))
    from_this_person_to_poi_and_poi.append((my_dataset[person]['from_this_person_to_poi'] != "NaN") & (my_dataset[person]['poi'] == True))
    other_and_poi.append((my_dataset[person]['other'] != "NaN") & (my_dataset[person]['poi'] == True))
    salary_and_poi.append((my_dataset[person]['salary'] != "NaN") & (my_dataset[person]['poi'] == True))
    shared_receipt_with_poi_and_poi.append((my_dataset[person]['shared_receipt_with_poi'] != "NaN") & (my_dataset[person]['poi'] == True))
    to_messages_and_poi.append((my_dataset[person]['to_messages'] != "NaN") & (my_dataset[person]['poi'] == True))
    total_payments_and_poi.append((my_dataset[person]['total_payments'] != "NaN") & (my_dataset[person]['poi'] == True))
    total_stock_value_and_poi.append((my_dataset[person]['total_stock_value'] != "NaN") & (my_dataset[person]['poi'] == True))
    bonus_and_poi.append((my_dataset[person]['bonus'] != "NaN") & (my_dataset[person]['poi'] == True))
print "email_address_and_poi" ,sum(email_address_and_poi)
print "exercised_stock_options_and_poi" ,sum(exercised_stock_options_and_poi)
print "expenses_and_poi" ,sum(expenses_and_poi)
print "from_messages_and_poi" ,sum(from_messages_and_poi)
print "from_poi_to_this_person_and_poi" ,sum(from_poi_to_this_person_and_poi)
print "from_this_person_to_poi_and_poi" ,sum(from_this_person_to_poi_and_poi)
print "other_and_poi" ,sum(other_and_poi)
print "salary_and_poi" ,sum(salary_and_poi)
print "shared_receipt_with_poi_and_poi" ,sum(shared_receipt_with_poi_and_poi)
print "to_messages_and_poi" ,sum(to_messages_and_poi)
print "total_payments_and_poi" ,sum(total_payments_and_poi)
print "total_stock_value_and_poi" ,sum(total_stock_value_and_poi)
print "bonus_and_poi" ,sum(bonus_and_poi)

deferral_income_and_poi 11
director_fees_and_poi 0
email_address_and_poi 18
exercised_stock_options_and_poi 12
expenses_and_poi 18
from_messages_and_poi 14
from_poi_to_this_person_and_poi 14
from_this_person_to_poi_and_poi 14
loan_advances_and_poi 1
long_term_incentive_and_poi 12
other_and_poi 18
restricted_stock_deferred_and_poi 0
salary_and_poi 17
shared_receipt_with_poi_and_poi 14
to_messages_and_poi 14
total_payments_and_poi 18
total_stock_value_and_poi 18
bonus_and_poi 16
